# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_csv_file = "output_data/cities.csv"

weather_df = pd.read_csv(weather_csv_file)

weather_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Sioux Lookout,75,CA,1586541570,50,50.10,-91.92,30.2,4.70
1,Hilo,90,US,1586541577,73,19.73,-155.09,72.0,3.36
2,Barrow,75,US,1586541577,91,71.29,-156.79,-14.8,5.82
3,Udachny,12,RU,1586541577,62,66.42,112.40,24.8,8.95
4,Port Elizabeth,77,ZA,1586541578,88,-33.92,25.57,62.6,9.17


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key.
gmaps.configure(api_key=g_key)

# Convert Humidity to float and store
# Also, handle NaN values
weather_df = weather_df.dropna()
humidity = weather_df["Humidity"].astype(float)

# Store 'Latitude' and 'Longitude' into  locations. 
locations = weather_df[["Lat", "Lng"]].astype(float)

# Create a humidity Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=max(weather_df["Humidity"]),
                                 point_radius = 2)

fig.add_layer(heat_layer)

print(fig)

Figure(layout=FigureLayout(height='420px'))


### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [19]:
# A max temperature lower than 80 degrees but higher than 70.
idealweather_df = weather_df.loc[weather_df["Max Temp"]<80]
idealweather_df = weather_df.loc[weather_df["Max Temp"]>70]
# Wind speed less than 10 mph.
idealweather_df = weather_df.loc[weather_df["Wind Speed"]<10]
#  Zero cloudiness.
idealweather_df = weather_df.loc[weather_df["Cloudiness"]==0]

idealweather_df.dropna

idealweather_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
11,Qaanaaq,0,GL,1586541579,83,77.48,-69.36,8.28,1.68
42,São Filipe,0,CV,1586541583,69,14.90,-24.50,76.95,12.86
49,Maragogi,0,BR,1586541584,62,-9.01,-35.22,86.04,10.87
70,Port Hedland,0,AU,1586541588,88,-20.32,118.57,73.40,6.93
85,Lunglei,0,IN,1586541590,45,22.88,92.73,63.75,2.15


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [22]:
hotel_df=idealweather_df


params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    # print(json.dumps(response, indent=4, sort_keys=True))
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 11: Qaanaaq.
Closest hotel is Qaanaaq Hotel.
------------
Retrieving Results for Index 42: São Filipe.
Closest hotel is Tortuga B&B.
------------
Retrieving Results for Index 49: Maragogi.
Closest hotel is Hotel Praia Dourada.
------------
Retrieving Results for Index 70: Port Hedland.
Closest hotel is The Esplanade Hotel.
------------
Retrieving Results for Index 85: Lunglei.
Closest hotel is Hotel Elite.
------------
Retrieving Results for Index 114: Montale.
Closest hotel is Albergo Il Cochino.
------------
Retrieving Results for Index 130: Awjilah.
Missing field/result... skipping.
------------
Retrieving Results for Index 146: Leningradskiy.
Missing field/result... skipping.
------------
Retrieving Results for Index 147: Bagdarin.
Missing field/result... skipping.
------------
Retrieving Results for Index 160: San Luis.
Closest hotel is Huarpes Hotel.
------------
Retrieving Results for Index 164: Talaya.
Missing field/result... skipping.
------------


In [24]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [28]:
# Assign the marker layer to a variable
locations = hotel_df[["Lat", "Lng"]]
markers = gmaps.marker_layer(locations)

# Create hotel symbol layer
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=hotel_info
)
# Add the layer to the map
fig.add_layer(markers)
fig.add_layer(hotel_layer)
fig

Figure(layout=FigureLayout(height='420px'))